## **Generate SQL-files**

In [1]:
from visiosql import SqlGen
import os

file = 'diagram.vsdx'
gen = SqlGen(file, auto_extract=True, schema='dbo')

file_counts = []
for folder in ['tables', 'constraints']:
    folder_path = os.getcwd() + '\\scripts\\' + folder + '\\'
    file_count = gen.create_files(folder, folder_path)
    file_counts.append(file_count)

print('Database creation files created successfully!', 'Total files created:', sum(file_counts))

tables : 4
constraints : 3
Database creation files created successfully! Total files created: 7


## **Execute SQL<span style="color: #000000;">-</span>files**

In [1]:
import pyodbc
import glob
import os
from secret import conn_str

def get_sql(fname):
    with open(fname, 'r') as f: 
        return f.read()

conn = pyodbc.connect(conn_str)
errs = 0
print('Executing files...')
for folder in ['tables', 'constraints']:
    folder_path = os.getcwd() + '\\scripts\\' + folder + '\\'
    print(folder, ':', len(os.listdir(folder_path)))
    for filepath in glob.iglob(folder_path + '*.sql'):
        try:
            conn.execute(get_sql(filepath))
        except Exception as e:
            errs += 1
            print('Error', errs, ':', filepath, e)
if errs:
    print('Execution failed!', 'Errors in', errs, 'files found.')
else:
    conn.commit()
    print('Database ojects created succesfully!')


Executing files...
tables : 4
constraints : 3
Database ojects created succesfully!


## **Visualize Relationships**

In [ ]:
import pyodbc
import graphviz
import os
from secret import conn_str

os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'
g = graphviz.Graph('G', filename='relationships.gv')

conn = pyodbc.connect(conn_str)
cur = conn.execute("""
SELECT p.Name, r.Name FROM Person p
    JOIN Residence r ON p.ResidenceId=r.ResidenceId
UNION
SELECT p.Name, f.Name FROM Person p
    JOIN Family f ON p.FamilyId=f.FamilyId
""")
rows = cur.fetchall()

for row in rows:
    parent = row[0]
    child = row[1]
    g.node(parent, labelloc='b', image=f'images\\{parent}.png')
    g.node(child, labelloc='b', image=f'images\\{child}.png')
    g.edge(parent, child)

g.view()